In [5]:
import pandas as pd
import numpy as np

In [9]:
df_dict = {'IRI VS POL_0316': 5, 'ITA VS FRA_0316': 3, 'ITA VS NED_0316':4, 'ITA VS POL_0316': 3, 'JPA VS FRA_0316': 3, 'USA VS BRA_0316': 4, 'USA VS FRA_0316': 5, 'USA VS POL_0316': 3}

In [10]:
def combine_df(df_dict):
    game = 1
    col = ['Rally', 'Team', 'No.', 'Space', 'Action', 'Errors', 'Attempts', 'Score']
    combined_df = pd.DataFrame(columns = ['Game'] + col)
    for key, value in df_dict.items():
        for i in range(1, value+1):
            try:
                df = pd.read_excel('data/'+key+'.xlsx', dtype={'Rally': float, 'No.': str, 'Action': str}, sheet_name='Set'+str(i))
                df = df[col]
                df = df.drop(index=len(df)-1, axis=0)    # 最後一列為總比分
                df.insert(0, 'Game', game)
                combined_df = pd.concat([combined_df, df])
                game += 1
            except Exception as e:
                print('data/'+key+'.xlsx', 'Set'+str(i))
                print(e, '\n', '='*30)
    
    combined_df.insert(8, 'Nothing', np.nan)
    combined_df = combined_df.reset_index(drop=True)
    return combined_df

In [33]:
df = combine_df(df_dict)

In [34]:
df = df.drop(index=df[df['Space'].isna()].index)    # 把 OP 或一些其他的 noise data 去除

# 合併 Attempt 和 Action、加上 Nothing label
for idx, row in df.iterrows():
    if(pd.isna(row['Attempts'])):
        df.loc[idx, 'Action'] = row['Action'] + '_0'
    else:
        df.loc[idx, 'Action'] = row['Action'] + '_1'
    if(pd.isna(row['Errors']) and pd.isna(row['Score'])):
        df.loc[idx, 'Nothing'] = 1.0

df = df.drop(columns='Attempts')
df = df.reset_index(drop=True)
df = df.replace({' ': np.nan})

In [35]:
# 檢查是否有異常值 or 空值
# team_list = ['IRI', 'POL', 'ITA', 'FRA', 'NED', 'JPA', 'USA', 'BRA']
# space_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 'FZ', 'L', 'M', 'Y']
# action_list = ['JS', 'SS', 'FS', 'R', 'G', 'A', 'B', 'D', 'OP']

In [36]:
rally_data = []
for d in df.groupby(['Game', 'Rally']):
    rally_data.append(d[1])
print(len(rally_data))

1292


In [37]:
import random as rd

In [38]:
rd.shuffle(rally_data)
shuffled_df = pd.DataFrame(columns=df.columns)

for rally in rally_data:
    shuffled_df = pd.concat([shuffled_df, rally])

In [39]:
shuffled_df

,Game,Rally,Team,No.,Space,Action,Errors,Nothing,Score
5007,21,25.0,USA,15,Y,JS_0,NaN,1.0,NaN
5008,21,25.0,BRA,18,7,R_0,NaN,1.0,NaN
5009,21,25.0,BRA,1,2,G_0,NaN,1.0,NaN
5010,21,25.0,BRA,23,2,A_0,NaN,NaN,1.0
5011,21,25.0,USA,8,8,D_0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3539,15,30.0,POL,20,L,JS_0,NaN,1.0,NaN
3540,15,30.0,ITA,15,8,R_0,NaN,1.0,NaN
3541,15,30.0,ITA,6,2,G_0,NaN,1.0,NaN
3542,15,30.0,ITA,5,1,At_0,NaN,NaN,1.0


In [41]:
shuffled_df.groupby('Score').count()

,Game,Rally,Team,No.,Space,Action,Errors,Nothing
Score,,,,,,,,
1.0,891,891,890,891,891,891,0,0


In [42]:
shuffled_df.groupby('Errors').count()

,Game,Rally,Team,No.,Space,Action,Nothing,Score
Errors,,,,,,,,
1.0,1288,1288,1287,1288,1288,1288,0,0


In [43]:
shuffled_df.to_csv('test.csv', index=False)